# Install library (Detectron2)

In [ ]:
!pip install opencv-python-headless pandas

!pip install torch torchvision

!pip install 'git+https://github.com/facebookresearch/detectron2.git'

# Imports and utility functions

In [ ]:
import os
import cv2
import math
import time
import argparse
import numpy as np
import pandas as pd
from datetime import timedelta
import json
import torch

from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import Visualizer, ColorMode

from IPython.display import HTML, display
from base64 import b64encode


def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def time_from_frame(frame_idx: int, fps: float) -> float:
    if not fps or math.isclose(fps, 0.0):
        fps = 30.0
    return frame_idx / fps

def fmt_hms(seconds: float) -> str:
    return str(timedelta(seconds=float(seconds))).split(".")[0]

def build_cfg(config_path: str, weights_path: str, score_thresh: float = 0.5, device: str = "cuda"):
    cfg = get_cfg()
    cfg.merge_from_file(config_path)
    cfg.MODEL.WEIGHTS = weights_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_thresh

    # Auto-select device
    if device.lower() in ["cuda", "gpu"] and torch.cuda.is_available():
        cfg.MODEL.DEVICE = "cuda"
        print("--- Configuration using: GPU (cuda) ---")
    else:
        cfg.MODEL.DEVICE = "cpu"
        print("--- Configuration using: CPU ---")
    return cfg

def alpha_blend_mask(image, masks, colors=None, alpha=0.45):
    """
    Draw a semi-transparent mask (Fast, avoids Visualizer if optimizing for FPS).
    masks: Tensor/ndarray [N, H, W] (bool or 0/1)
    colors: list of BGR tuples, len=N
    """
    if masks is None or len(masks) == 0:
        return image
    if isinstance(masks, torch.Tensor):
        masks = masks.detach().cpu().numpy()
    overlay = image.copy()
    N = masks.shape[0]
    H, W = image.shape[:2]

    if colors is None:
        rng = np.random.default_rng(123)
        colors = [tuple(int(c) for c in rng.integers(64, 255, size=3)) for _ in range(N)]  # BGR

    for i in range(N):
        m = masks[i].astype(bool)
        if m.sum() == 0:
            continue
        color = colors[i % len(colors)]
        overlay[m] = (0.6 * overlay[m] + 0.4 * np.array(color)).astype(np.uint8)

    out = (alpha * overlay + (1 - alpha) * image).astype(np.uint8)
    return out

# --- Slot Drawing Utilities ---
def load_and_scale_slots(slots_json_path, target_w, target_h):
    """Load and scale polygons from JSON file."""
    if not os.path.isfile(slots_json_path):
        print(f"⚠️  Slots JSON file not found: {slots_json_path}")
        return [], None, None

    with open(slots_json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    slots = data.get("slots", [])
    ref_w, ref_h = data.get("frame_size", [None, None])

    # Scale polygons if reference size differs from video size
    if ref_w and ref_h and (ref_w != target_w or ref_h != target_h):
        print(f"--- Scaling slots from {ref_w}x{ref_h} to {target_w}x{target_h}")
        sx, sy = target_w / float(ref_w), target_h / float(ref_h)
        slots = [
            [[int(px * sx), int(py * sy)] for (px, py) in poly]
            for poly in slots
        ]

    return slots

def create_slot_masks(slots, height, width):
    """
    Convert a list of polygons (slots) into a stack of binary masks.
    """
    if not slots:
        return None, None

    num_slots = len(slots)
    slot_masks = np.zeros((num_slots, height, width), dtype=np.uint8)
    slot_areas = []

    for i, poly in enumerate(slots):
        pts = np.array(poly, dtype=np.int32)
        mask_i = np.zeros((height, width), dtype=np.uint8)
        cv2.fillPoly(mask_i, [pts], 1)
        slot_masks[i] = mask_i
        slot_areas.append(np.sum(mask_i))

    return slot_masks, np.array(slot_areas)

def check_slot_occupancy(detection_masks, slot_masks, slot_areas, threshold=0.2):
    """
    Check which slots are occupied based on % intersection area.
    - detection_masks: [N, H, W] (bool/uint8), N is the number of objects
    ...
    """
    num_slots = slot_masks.shape[0]

    if num_slots == 0:
        return []

    # If no objects are detected, all slots are empty
    if detection_masks is None or len(detection_masks) == 0:
        return [False] * num_slots

    occupied_flags = [False] * num_slots

    # Combine all object masks into one large mask [H, W]
    combined_detection_mask = np.any(detection_masks, axis=0).astype(np.uint8)

    for i in range(num_slots):
        slot_area_i = slot_areas[i]
        if slot_area_i == 0:
            continue

        # Calculate intersection
        intersection = np.logical_and(slot_masks[i], combined_detection_mask).astype(np.uint8)
        intersection_area = np.sum(intersection)

        ratio = intersection_area / slot_area_i

        if ratio >= threshold:
            occupied_flags[i] = True

    return occupied_flags

def draw_slots_on_frame(frame, slots, occupied_flags=None,
                        color_empty_bgr=(0, 128, 0),   # Green (empty)
                        color_occupied_bgr=(0, 0, 128), # Red (occupied)
                        thickness=2, draw_label=True):
    """Draw loaded polygons (slots) onto the frame, with different colors."""

    if occupied_flags is None:
        occupied_flags = [False] * len(slots)

    for i, poly in enumerate(slots, start=1):
        pts = np.array(poly, dtype=np.int32)

        is_occupied = occupied_flags[i-1]
        color = color_occupied_bgr if is_occupied else color_empty_bgr

        cv2.polylines(frame, [pts], True, color, thickness)

        if draw_label:
            cx = int(np.mean(pts[:, 0]))
            cy = int(np.mean(pts[:, 1]))
            cv2.putText(frame, str(i), (cx - 10, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 255), 2)
    return frame

print("✅ Utility functions loaded successfully.")

# Main processing function process_video

In [ ]:
def process_video(
        config_path: str,
        weights_path: str,
        input_video: str,
        out_dir: str,
        score_thresh: float = 0.5,
        use_visualizer: bool = False,
        mask_alpha: float = 0.45,
        slots_json_path: str = None,
        draw_slots: bool = False
):
    assert os.path.isfile(input_video), f"Video not found: {input_video}"
    ensure_dir(out_dir)

    OCCUPANCY_THRESHOLD = 0.2

    cfg = build_cfg(config_path, weights_path, score_thresh=score_thresh, device="cuda")
    predictor = DefaultPredictor(cfg)

    dataset_name = cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) > 0 else None
    meta = MetadataCatalog.get(dataset_name) if dataset_name else None
    thing_classes = getattr(meta, "thing_classes", None)

    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open video: {input_video}")

    input_fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
    output_fps = 30.0

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"--- Reading video: {input_video}")
    print(f"--- Specs (Input): {width}x{height} @ {input_fps:.2f} FPS, Total: {total_frames} frames")
    print(f"--- Specs (Output): {output_fps:.2f} FPS")

    slots = []
    slot_masks = None
    slot_areas = None
    if draw_slots and slots_json_path:
        slots = load_and_scale_slots(slots_json_path, width, height)
        if not slots:
            print(f"Warning: Could not load/scale slots. Slots will not be drawn.")
            draw_slots = False
        else:
            slot_masks, slot_areas = create_slot_masks(slots, height, width)
            if slot_masks is not None:
                print(f"--- Created {len(slots)} slot masks.")
            else:
                print("Warning: Could not create slot masks.")
                draw_slots = False

    elif draw_slots and not slots_json_path:
        print("Warning: --draw_slots is enabled but --slots_json was not provided. Slots will not be drawn.")
        draw_slots = False

    SLOT_THICKNESS = 2
    SLOT_DRAW_LABEL = True

    base = os.path.splitext(os.path.basename(input_video))[0]
    out_video = os.path.join(out_dir, f"{base}_detect_overlay.avi")
    out_csv = os.path.join(out_dir, f"{base}_seg.csv")
    out_sum = os.path.join(out_dir, f"{base}_summary.txt")

    writer = cv2.VideoWriter(out_video, cv2.VideoWriter_fourcc(*"XVID"), output_fps, (width, height))

    rows = []
    frames_with_detections = 0
    frame_idx = 0

    t0 = time.time()
    while True:
        ok, frame = cap.read()
        if not ok:
            break

        outputs = predictor(frame)
        instances = outputs["instances"].to("cpu")

        filtered_masks = None
        det_count = 0

        if len(instances) > 0:
            classes = instances.pred_classes.numpy()
            scores = instances.scores.numpy()
            boxes = instances.pred_boxes.tensor.numpy()
            masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None

            filtered_masks = masks
            det_count = len(classes)

            if det_count > 0:
                frames_with_detections += 1

                if use_visualizer:
                    v = Visualizer(frame[:, :, ::-1], metadata=meta, instance_mode=ColorMode.IMAGE)
                    vis = v.draw_instance_predictions(instances).get_image()[:, :, ::-1]
                    frame_out = vis
                else:
                    frame_out = frame.copy()
                    if filtered_masks is not None:
                        frame_out = alpha_blend_mask(frame_out, filtered_masks, alpha=mask_alpha)

                    for i in range(det_count):
                        x1, y1, x2, y2 = boxes[i].astype(int)
                        score = float(scores[i])
                        cls_id = int(classes[i])
                        cname = thing_classes[cls_id] if (thing_classes and 0 <= cls_id < len(thing_classes)) else str(
                            cls_id)

                        cv2.rectangle(frame_out, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame_out, f"{cname} {score:.2f}", (x1, max(0, y1 - 6)),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

                        t_sec = time_from_frame(frame_idx, input_fps)
                        area = int(filtered_masks[i].sum()) if filtered_masks is not None else -1
                        rows.append({
                            "video": base,
                            "frame": frame_idx,
                            "time_sec": round(t_sec, 3),
                            "time_hms": fmt_hms(t_sec),
                            "class_id": cls_id,
                            "class_name": cname,
                            "score": round(score, 4),
                            "x1": int(x1), "y1": int(y1), "x2": int(x2), "y2": int(y2),
                            "mask_area_px": area,
                            "img_w": width, "img_h": height
                        })
            else:
                frame_out = frame
        else:
            frame_out = frame

        # Calculate empty slot count
        empty_slot_count = 0
        total_slot_count = len(slots) if slots else 0
        occupied_spaces = 0 # Default
        occupied_flags = None

        if draw_slots and slot_masks is not None:
            occupied_flags = check_slot_occupancy(
                filtered_masks,
                slot_masks,
                slot_areas,
                threshold=OCCUPANCY_THRESHOLD
            )
            empty_slot_count = occupied_flags.count(False)
            occupied_spaces = total_slot_count - empty_slot_count # Calculate occupied slots
        elif draw_slots:
            empty_slot_count = total_slot_count
            occupied_flags = [False] * total_slot_count

        # Draw the slots
        if draw_slots and slots:
            frame_out = draw_slots_on_frame(
                frame_out,
                slots,
                occupied_flags=occupied_flags,
                thickness=SLOT_THICKKNOWN,
                draw_label=SLOT_DRAW_LABEL
            )

        hud_text = f"Car_in_park: {det_count}"

        # Add slot info if available
        if draw_slots and slots:
            hud_text += f" | Free_spaces: {empty_slot_count} | Occupied_spaces: {occupied_spaces}"

        cv2.putText(frame_out, hud_text,
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2,
                    cv2.LINE_AA)

        writer.write(frame_out)
        frame_idx += 1

        if frame_idx % 100 == 0:
            print(f"Processed frame {frame_idx}/{total_frames}")

    cap.release()
    writer.release()

    df = pd.DataFrame(rows, columns=[
        "video", "frame", "time_sec", "time_hms", "class_id", "class_name", "score",
        "x1", "y1", "x2", "y2", "mask_area_px", "img_w", "img_h"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")

    elapsed = time.time() - t0
    ratio = (frames_with_detections / max(1, total_frames)) * 100.0
    with open(out_sum, "w", encoding="utf-8") as f:
        f.write(f"Video: {base}\n")
        f.write(f"FPS(input): {input_fps:.2f}\n")
        f.write(f"FPS(output): {output_fps:.2f}\n")
        f.write(f"Frames: {total_frames}\n")
        f.write(f"Frames with detections: {frames_with_detections} ({ratio:.2f}%)\n")
        f.write(f"Processing time: {elapsed:.2f}s\n")
        f.write(f"Output video: {out_video}\n")
        f.write(f"CSV: {out_csv}\n")

    print("✅ Done")
    print("   →", out_video)
    print("   →", out_csv)
    print("   →", out_sum)

    return out_video

print("✅ process_video function loaded successfully.")

# Run processing and display video

In [ ]:
OUTPUT_DIR = "/content/seg_outputs"

# Video 1
CONFIG_PATH_1 = "/content/config.yaml"
WEIGHTS_PATH_1 = "/content/model_final.pth"
VIDEO_INPUT_PATH_1 = "/content/7252.mp4"
SLOTS_JSON_PATH_1 = "/content/7252_slots.json"

# Video 2
CONFIG_PATH_2 = "/content/config.yaml"
WEIGHTS_PATH_2 = "/content/model_final.pth"
VIDEO_INPUT_PATH_2 = "/content/7254.mp4"
SLOTS_JSON_PATH_2 = "/content/7252_slots.json"

print(f"🚀 Starting processing for VIDEO 1: {VIDEO_INPUT_PATH_1}")
# Call the processing function for video 1
output_avi_path_1 = process_video(
    config_path=CONFIG_PATH_1,
    weights_path=WEIGHTS_PATH_1,
    input_video=VIDEO_INPUT_PATH_1,
    out_dir=OUTPUT_DIR,
    score_thresh=0.5,
    use_visualizer=False,
    slots_json_path=SLOTS_JSON_PATH_1,
    draw_slots=True if SLOTS_JSON_PATH_1 else False
)

output_mp4_path_1 = os.path.splitext(output_avi_path_1)[0] + ".mp4"
print(f"\n--- Converting {output_avi_path_1} to {output_mp4_path_1} for display ---")
!ffmpeg -i $output_avi_path_1 -vcodec libx264 $output_mp4_path_1 -y -loglevel quiet

print("✅ Conversion complete. Displaying Video 1:")
mp4_1 = open(output_mp4_path_1,'rb').read()
data_url_1 = "data:video/mp4;base64," + b64encode(mp4_1).decode()
display(HTML(f"""
<video width=800 controls autoplay loop>
      <source src="{data_url_1}" type="video/mp4">
</video>
"""))

print(f"🚀 Starting processing for VIDEO 2: {VIDEO_INPUT_PATH_2}")
# Call the processing function for video 2
output_avi_path_2 = process_video(
    config_path=CONFIG_PATH_2,
    weights_path=WEIGHTS_PATH_2,
    input_video=VIDEO_INPUT_PATH_2,
    out_dir=OUTPUT_DIR,
    score_thresh=0.5,
    use_visualizer=False,
    slots_json_path=SLOTS_JSON_PATH_2,
    draw_slots=True if SLOTS_JSON_PATH_2 else False
)

output_mp4_path_2 = os.path.splitext(output_avi_path_2)[0] + ".mp4"
print(f"\n--- Converting {output_avi_path_2} to {output_mp4_path_2} for display ---")
!ffmpeg -i $output_avi_path_2 -vcodec libx264 $output_mp4_path_2 -y -loglevel quiet

print("✅ Conversion complete. Displaying Video 2:")
mp4_2 = open(output_mp4_path_2,'rb').read()
data_url_2 = "data:video/mp4;base64," + b64encode(mp4_2).decode()
display(HTML(f"""
<video width=800 controls autoplay loop>
      <source src="{data_url_2}" type="video/mp4">
</video>
"""))

# Compress and load results

In [ ]:
print(f"Processing complete!")
print(f"Results have been saved to: {OUTPUT_DIR}")

ZIP_PATH = "/content/seg_outputs.zip"

print(f"Compressing results to {ZIP_PATH}...")

shutil.make_archive(
    base_name=ZIP_PATH.replace(".zip", ""),
    format='zip',
    root_dir=OUTPUT_DIR
)

print(f"Compression complete! Preparing for download...")

files.download(ZIP_PATH)